#### <center> <h1 style="color:blue; display: inline-block">Assignment 2</h1> </center><br>
<h2 style="color:blue; display: inline-block">Subject: </h2> 
<h2 style="color:black; display: inline-block">Introduction to data sciences</h2> <br>
<h2 style="color:blue; display: inline-block">Group members:</h2> <br>
<h3 style="color:blue; display: inline-block">NAME:</h3> 
<h3 style="color:black; display: inline-block; margin-left: 40px">AMBER RIAZ</h3> 
<h3 style="color:blue; display: inline-block; margin-left: 35px">Registration number:</h3> 
<h3 style="color:black; display: inline-block">FA18-BCS-035</h3> <br>
<h3 style="color:blue; display: inline-block">NAME:</h3> 
<h3 style="color:black; display: inline-block; margin-left: 40px">SAAIM SIDDIQUI</h3> 
<h3 style="color:blue; display: inline-block; margin-left: 5px">Registration number:</h3> 
<h3 style="color:black; display: inline-block">FA18-BCS-087</h3> <br>


# Importing libraries


In [7]:
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics #to calculate accuracy 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
from nltk import PorterStemmer
import pickle 
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Loading the data 


In [8]:
sample_data_t = pd.read_csv("train.csv")
# sample_data_t.columns = ['index', 'comment', 'output'] #Giving column headers to the dataset
sample_data_t = sample_data_t.dropna(axis = 0, how = 'any')
sample_data_t = sample_data_t.drop(columns = ['Index'])
sample_data_t.head()

,comment,output
0,time to eat with my best buddy! #lunch,Happy
1,@user @user if they want reflection money. #ksleg,Happy
2,---Good job but I’ will expect a lot more in f...,Happy
3,totally dissatisfied with the service###%%@@ n...,Sad
4,loved my work!!!!!,Happy


# Encoding the output 

In [9]:
#initializing the label encoder

output = pd.DataFrame({"output":["Happy","Sad"]})
output_label_encoder = LabelEncoder()

#training the label encoder
output_label_encoder.fit(np.ravel(output))

LabelEncoder()

In [10]:


# Transform Output of into Numerical Representation

print("\n\output Attribute After Label Encoding:")
print("========================================\n")
sample_data_t["encoded_output"] = output_label_encoder.transform(sample_data_t['output'])
print(sample_data_t[["output", "encoded_output"]])
sample_data_t.head()


\output Attribute After Label Encoding:

   output  encoded_output
0   Happy               0
1   Happy               0
2   Happy               0
3     Sad               1
4   Happy               0
5     Sad               1
6   Happy               0
7     Sad               1
8   Happy               0
9     Sad               1
10    Sad               1
11  Happy               0
12    Sad               1
13  Happy               0
14  Happy               0


,comment,output,encoded_output
0,time to eat with my best buddy! #lunch,Happy,0
1,@user @user if they want reflection money. #ksleg,Happy,0
2,---Good job but I’ will expect a lot more in f...,Happy,0
3,totally dissatisfied with the service###%%@@ n...,Sad,1
4,loved my work!!!!!,Happy,0


In [11]:
sample_data_t["output"] = sample_data_t["encoded_output"]
sample_data_t.drop('encoded_output', inplace= True, axis= 1)
sample_data_t.head()

,comment,output
0,time to eat with my best buddy! #lunch,0
1,@user @user if they want reflection money. #ksleg,0
2,---Good job but I’ will expect a lot more in f...,0
3,totally dissatisfied with the service###%%@@ n...,1
4,loved my work!!!!!,0


# Removing twitter handlers (@user)

In [12]:
#handling twitter handles @user 

def remove_pattern(text,pattern):
    
    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)
    
    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)
    
    return text


sample_data_t['Tidy_Tweets'] = np.vectorize(remove_pattern)(sample_data_t['comment'], "@[\w]*")
# sample_data_t['Tidy_Tweets'] = np.vectorize(remove_pattern)(sample_data_t['tweet'], "@[\w]*")

sample_data_t.head(10)

,comment,output,Tidy_Tweets
0,time to eat with my best buddy! #lunch,0,time to eat with my best buddy! #lunch
1,@user @user if they want reflection money. #ksleg,0,if they want reflection money. #ksleg
2,---Good job but I’ will expect a lot more in f...,0,---Good job but I’ will expect a lot more in f...
3,totally dissatisfied with the service###%%@@ n...,1,totally dissatisfied with the service###%% nev...
4,loved my work!!!!!,0,loved my work!!!!!
5,Worst customer care service......@@$$$angry,1,Worst customer care service......$$$angry
6,Brilliant effort guys!!!,0,Brilliant effort guys!!!
7,@user @user you point one finger @user million...,1,you point one finger millions are pointed r...
8,"words r free, it's how u use that can cost you...",0,"words r free, it's how u use that can cost you..."
9,you might be a libtard if... #libtard #sjw #li...,1,you might be a libtard if... #libtard #sjw #li...


# Removing punctuations 

In [13]:
sample_data_t['Tidy_Tweets'] = sample_data_t['Tidy_Tweets'].str.replace("[^a-zA-Z#]", " ")

sample_data_t.head(10)

,comment,output,Tidy_Tweets
0,time to eat with my best buddy! #lunch,0,time to eat with my best buddy #lunch
1,@user @user if they want reflection money. #ksleg,0,if they want reflection money #ksleg
2,---Good job but I’ will expect a lot more in f...,0,Good job but I will expect a lot more in f...
3,totally dissatisfied with the service###%%@@ n...,1,totally dissatisfied with the service### nev...
4,loved my work!!!!!,0,loved my work
5,Worst customer care service......@@$$$angry,1,Worst customer care service angry
6,Brilliant effort guys!!!,0,Brilliant effort guys
7,@user @user you point one finger @user million...,1,you point one finger millions are pointed r...
8,"words r free, it's how u use that can cost you...",0,words r free it s how u use that can cost you...
9,you might be a libtard if... #libtard #sjw #li...,1,you might be a libtard if #libtard #sjw #li...


# Removing short words 


In [14]:
sample_data_t['Tidy_Tweets'] = sample_data_t['Tidy_Tweets'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

sample_data_t.head(10)

,comment,output,Tidy_Tweets
0,time to eat with my best buddy! #lunch,0,time with best buddy #lunch
1,@user @user if they want reflection money. #ksleg,0,they want reflection money #ksleg
2,---Good job but I’ will expect a lot more in f...,0,Good will expect more future
3,totally dissatisfied with the service###%%@@ n...,1,totally dissatisfied with service### never use...
4,loved my work!!!!!,0,loved work
5,Worst customer care service......@@$$$angry,1,Worst customer care service angry
6,Brilliant effort guys!!!,0,Brilliant effort guys
7,@user @user you point one finger @user million...,1,point finger millions pointed right back #jewi...
8,"words r free, it's how u use that can cost you...",0,words free that cost #verbal #abuse #love #adu...
9,you might be a libtard if... #libtard #sjw #li...,1,might libtard #libtard #sjw #liberal #politics


# Tokenizing tweets 

In [15]:
from nltk.tokenize import TweetTokenizer 


tokenizer = TweetTokenizer()
tokenized_tweet = sample_data_t['Tidy_Tweets'].apply(tokenizer.tokenize)

# tokenized_tweet = sample_data_t['Tidy_Tweets'].apply(lambda x: x.split())

tokenized_tweet.head(10)

0                    [time, with, best, buddy, #lunch]
1              [they, want, reflection, money, #ksleg]
2                   [Good, will, expect, more, future]
3    [totally, dissatisfied, with, service, #, #, #...
4                                        [loved, work]
5              [Worst, customer, care, service, angry]
6                            [Brilliant, effort, guys]
7    [point, finger, millions, pointed, right, back...
8    [words, free, that, cost, #verbal, #abuse, #lo...
9    [might, libtard, #libtard, #sjw, #liberal, #po...
Name: Tidy_Tweets, dtype: object

# Stemming 

In [16]:
ps = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])

tokenized_tweet.head(10)

0                    [time, with, best, buddi, #lunch]
1                 [they, want, reflect, money, #ksleg]
2                    [good, will, expect, more, futur]
3    [total, dissatisfi, with, servic, #, #, #, nev...
4                                         [love, work]
5                 [worst, custom, care, servic, angri]
6                             [brilliant, effort, guy]
7    [point, finger, million, point, right, back, #...
8    [word, free, that, cost, #verbal, #abus, #love...
9     [might, libtard, #libtard, #sjw, #liber, #polit]
Name: Tidy_Tweets, dtype: object

In [17]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

sample_data_t['Tidy_Tweets'] = tokenized_tweet
sample_data_t.head(50)

,comment,output,Tidy_Tweets
0,time to eat with my best buddy! #lunch,0,time with best buddi #lunch
1,@user @user if they want reflection money. #ksleg,0,they want reflect money #ksleg
2,---Good job but I’ will expect a lot more in f...,0,good will expect more futur
3,totally dissatisfied with the service###%%@@ n...,1,total dissatisfi with servic # # # never use t...
4,loved my work!!!!!,0,love work
5,Worst customer care service......@@$$$angry,1,worst custom care servic angri
6,Brilliant effort guys!!!,0,brilliant effort guy
7,@user @user you point one finger @user million...,1,point finger million point right back #jewishs...
8,"words r free, it's how u use that can cost you...",0,word free that cost #verbal #abus #love #adult...
9,you might be a libtard if... #libtard #sjw #li...,1,might libtard #libtard #sjw #liber #polit


# Extracting features 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=2, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(sample_data_t['Tidy_Tweets'])

df_bow = pd.DataFrame(bow.todense())

df_bow.head(10)

,0,1
0,0,0
1,0,0
2,1,0
3,0,2
4,0,0
5,0,1
6,0,0
7,0,0
8,0,0
9,0,0


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=3, stop_words='english')

tfidf_matrix=tfidf.fit_transform(sample_data_t['Tidy_Tweets'])

df_tfidf = pd.DataFrame(tfidf_matrix.todense())

df_tfidf

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0
5,0.0,0.0,1.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,1.0,0.0
9,0.0,0.0,0.0


In [20]:
train_bow = bow
train_bow.todense()

matrix([[0, 0],
        [0, 0],
        [1, 0],
        [0, 2],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0]])

In [21]:
train_tfidf_matrix = tfidf_matrix
train_tfidf_matrix.todense()

matrix([[0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

# Splitting dataset into train-test

In [22]:
from sklearn.model_selection import train_test_split

sample_data_t.head()

,comment,output,Tidy_Tweets
0,time to eat with my best buddy! #lunch,0,time with best buddi #lunch
1,@user @user if they want reflection money. #ksleg,0,they want reflect money #ksleg
2,---Good job but I’ will expect a lot more in f...,0,good will expect more futur
3,totally dissatisfied with the service###%%@@ n...,1,total dissatisfi with servic # # # never use t...
4,loved my work!!!!!,0,love work


# Feature extraction by bow 

In [24]:
x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(train_bow,sample_data_t['output'],test_size=0.3,random_state=2)

# Feature extraction by Tf-IDF

In [25]:
x_train_tfidf, x_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split(train_tfidf_matrix,sample_data_t['output'],test_size=0.3,random_state=17)

# Training Random Forest by Bag-of-words approach

In [26]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators=10, max_features="sqrt", max_depth = None)
RF_model = RF_model.fit(x_train_bow,y_train_bow)

# Calculating predictions


In [29]:
from prettytable import PrettyTable
table = PrettyTable()
y_pred=RF_model.predict(x_test_bow)
table.field_names = ["*Labels*", "*predicted value*"] 
i = 0
while i < len(y_pred):
    table.add_row([y_test_bow.iloc[i],y_pred[i]])
    i = i+1
print(table)
print("Accuracy:",metrics.accuracy_score(y_test_bow, y_pred))

+----------+-------------------+
| *Labels* | *predicted value* |
+----------+-------------------+
|    0     |         0         |
|    0     |         0         |
|    1     |         0         |
|    0     |         0         |
|    1     |         0         |
+----------+-------------------+
Accuracy: 0.6


## Save the Trained Model

In [30]:
# Save the Model in a Pkl File

pickle.dump(RF_model, open('Assignment 2 IDS/RF_trained_model.pkl', 'wb'))

# Training decision tree classifiers 

In [31]:
from sklearn import tree 
DT_model = tree.DecisionTreeClassifier()
DT_model = DT_model.fit(x_train_bow,y_train_bow)

In [33]:
y_pred=DT_model.predict(x_test_bow)
table.field_names = ["*Labels*", "*predicted value*"] 
i = 0
while i < len(y_pred):
    table.add_row([y_test_bow.iloc[i],y_pred[i]])
    i = i + 1
print(table)
print("Accuracy:",metrics.accuracy_score(y_test_bow, y_pred))

+----------+-------------------+
| *Labels* | *predicted value* |
+----------+-------------------+
|    0     |         0         |
|    0     |         0         |
|    1     |         0         |
|    0     |         0         |
|    1     |         0         |
|    0     |         0         |
|    0     |         0         |
|    0     |         0         |
|    1     |         0         |
|    0     |         0         |
|    1     |         0         |
+----------+-------------------+
Accuracy: 0.6


## Save the Trained Model

In [34]:
# Save the Model in a Pkl File

pickle.dump(DT_model, open('Assignment 2 IDS/DT_trained_model.pkl', 'wb'))

## Load the Saved Model

In [35]:
# Load the Saved Model

model = pickle.load(open('Assignment 2 IDS/RF_trained_model.pkl', 'rb'))

In [36]:
tweet = input("\nPlease enter your tweet here : ")


Please enter your tweet here : This is a happy tweet


In [37]:
user_input = pd.DataFrame({"user_input":[tweet]})
user_input['tidy_tweet'] = np.vectorize(remove_pattern)(user_input['user_input'], "@[\w]*")
user_input['tidy_tweet'] = user_input['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
user_input['tidy_tweet'] = user_input['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


tokenizer = TweetTokenizer()
tokenized_tweet = user_input['tidy_tweet'].apply(tokenizer.tokenize)

ps = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

user_input['tidy_tweet'] = tokenized_tweet

bow_vectorizer = CountVectorizer(max_df=2, min_df=0.20, max_features=2, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(user_input['tidy_tweet'])

df_bow = pd.DataFrame(bow.todense())

df_bow

train_bow = bow
train_bow.todense()

pred= model.predict(train_bow)


if(pred == 0):
    print("happy")
else:
    print("sad")

happy
